In [33]:
import requests, itertools
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import pandas as pd

In [34]:
base_url = "https://www.lendingtree.com/student/refinance/"

In [35]:
url = f"{base_url}"
try:
    response = requests.get(url)
    response.raise_for_status()
except HTTPError as hp:
    print(hp)

In [36]:
page = BeautifulSoup(response.text, 'html.parser')
lender = page.findAll('div', {'class': 'lenderLogos'})
term = page.findAll('div', {'class': 'term'})
fixed = page.findAll('div', {'class': 'fixedAPR'})
variable = page.findAll('div', {'class': 'variableAPR'})
balance = page.findAll('div', {'class': 'balanceRange'})
credit = page.findAll('div', {'class': 'creditScore'})
detail = page.findAll('div', {'class': 'studentLenderDetails'})
loan_info_ls = []
loan_info = {}
for l, t, f, v, b, c, d in itertools.zip_longest(lender, term, fixed, variable, balance, credit, detail, fillvalue='value'):
    loan_info["Lender logo"] = l.img['data-lazy-src']
    loan_info["Term length (years)"] = t.span.text.strip()
    loan_info["Fixed APR"] = f.span.text.strip()
    loan_info["Variable APR"] = v.span.text.strip()
    loan_info["Balance range"] = b.span.text.strip()
    loan_info["Credit score"] = c.div.text.strip()
    loan_info["Detail information"] = d.text.strip()
    loan_info_ls.append(loan_info.copy())

In [37]:
len(loan_info_ls)

8

In [38]:
pd.DataFrame(loan_info_ls).to_json("studentloan.json", orient='records', lines=True)